In [1]:
import numpy as np
from blimpy import read_header, Waterfall, Filterbank

%matplotlib inline
import matplotlib.pyplot as plt

import sys, os, glob
sys.path.append("../../setigen")
import setigen as stg

tsamp = 1.0
fch1 = 6095.214842353016
df = -1.0e-06

fchans = 1024
tchans = 32

fs = np.arange(fch1, fch1 + fchans*df, df)
ts = np.arange(0, tchans*tsamp, tsamp)

In [3]:
# basic cnn training: 1
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

import numpy as np
import os, sys, errno
import time
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.2
# session = tf.Session(config=config)

def format_time(total):
    h = int(np.floor(total / 3600.))
    m = int(np.floor((total - 3600. * h) / 60.))
    s = (total - 3600. * h - 60. * m)
    return '%02dh %02dm %02.2fs' % (h, m, s)

start = time.time()
# dimensions of our images.
img_width, img_height = 32, 1024

VERSION = 'v1'

dir = 'scintillated_small'

train_data_dir = '/datax/scratch/bbrzycki/data/%s/train/' % (dir)
validation_data_dir = '/datax/scratch/bbrzycki/data/%s/validation/' % (dir)
nb_train_samples = 1000*2
nb_validation_samples = 200*2
epochs = 10
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model_dir = '/datax/scratch/bbrzycki/models/scintillated/'
try:
    os.makedirs(model_dir)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
model.save_weights(model_dir + '%s_%s.h5' % ('scintillated', VERSION))

end = time.time()
print('Saved model! Took %s' % format_time(end - start))

# at the end of 50 epochs: 31/31 - 16s 522ms/step
# loss: 0.1979 - acc: 0.9289 - val_loss: 0.1578 - val_acc: 0.9479

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 17s 535ms/step - loss: 0.7606 - acc: 0.5045 - val_loss: 0.6867 - val_acc: 0.5078
Epoch 2/10
31/31 [==============================] - 16s 509ms/step - loss: 0.6923 - acc: 0.5439 - val_loss: 0.6456 - val_acc: 0.6536
Epoch 3/10
31/31 [==============================] - 16s 522ms/step - loss: 0.6794 - acc: 0.6227 - val_loss: 0.5943 - val_acc: 0.6953
Epoch 4/10
31/31 [==============================] - 15s 490ms/step - loss: 0.6418 - acc: 0.6531 - val_loss: 0.6276 - val_acc: 0.6380
Epoch 5/10
31/31 [==============================] - 19s 610ms/step - loss: 0.6214 - acc: 0.6648 - val_loss: 0.5719 - val_acc: 0.7422
Epoch 6/10
31/31 [==============================] - 18s 592ms/step - loss: 0.6103 - acc: 0.6703 - val_loss: 0.7239 - val_acc: 0.5859
Epoch 7/10
31/31 [==============================] - 19s 617ms/step - loss: 0.6201 - acc: 0.6682 - val_loss: 0.5772 - va

In [6]:
# basic cnn training: 2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

import numpy as np
import os, sys, errno
import time
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.2
# session = tf.Session(config=config)

def format_time(total):
    h = int(np.floor(total / 3600.))
    m = int(np.floor((total - 3600. * h) / 60.))
    s = (total - 3600. * h - 60. * m)
    return '%02dh %02dm %02.2fs' % (h, m, s)

start = time.time()
# dimensions of our images.
img_width, img_height = 64, 1024

VERSION = 'v2'

dir = 'scintillated_small'

train_data_dir = '/datax/scratch/bbrzycki/data/%s/train/' % (dir)
validation_data_dir = '/datax/scratch/bbrzycki/data/%s/validation/' % (dir)
nb_train_samples = 1000*2
nb_validation_samples = 200*2
epochs = 10
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model_dir = '/datax/scratch/bbrzycki/models/scintillated/'
try:
    os.makedirs(model_dir)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
model.save_weights(model_dir + '%s_%s.h5' % ('scintillated', VERSION))

end = time.time()
print('Saved model! Took %s' % format_time(end - start))


Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/10
31/31 [==============================] - 64s 2s/step - loss: 0.6983 - acc: 0.5453 - val_loss: 0.6848 - val_acc: 0.5365
Epoch 2/10
31/31 [==============================] - 59s 2s/step - loss: 0.7525 - acc: 0.6130 - val_loss: 0.5821 - val_acc: 0.7786
Epoch 3/10
31/31 [==============================] - 58s 2s/step - loss: 0.6336 - acc: 0.7030 - val_loss: 0.6319 - val_acc: 0.6198
Epoch 4/10
31/31 [==============================] - 59s 2s/step - loss: 0.5479 - acc: 0.7625 - val_loss: 0.4761 - val_acc: 0.7760
Epoch 5/10
31/31 [==============================] - 59s 2s/step - loss: 0.5122 - acc: 0.7651 - val_loss: 0.4957 - val_acc: 0.7891
Epoch 6/10
31/31 [==============================] - 60s 2s/step - loss: 0.4703 - acc: 0.7928 - val_loss: 0.4811 - val_acc: 0.7448
Epoch 7/10
31/31 [==============================] - 57s 2s/step - loss: 0.4265 - acc: 0.7913 - val_loss: 0.5377 - val_acc: 0.7682
Epoch 8

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 126, 1022, 32)     896       
_________________________________________________________________
activation_16 (Activation)   (None, 126, 1022, 32)     0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 63, 511, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 509, 32)       9248      
_________________________________________________________________
activation_17 (Activation)   (None, 61, 509, 32)       0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 30, 254, 32)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 252, 64)       18496     
__________

In [13]:
# basic cnn training: 3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

import numpy as np
import os, sys, errno
import time
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.2
# session = tf.Session(config=config)

def format_time(total):
    h = int(np.floor(total / 3600.))
    m = int(np.floor((total - 3600. * h) / 60.))
    s = (total - 3600. * h - 60. * m)
    return '%02dh %02dm %02.2fs' % (h, m, s)

start = time.time()
# dimensions of our images.
img_width, img_height = 32, 1024

VERSION = 'v3'

dir = 'scintillated_small'

train_data_dir = '/datax/scratch/bbrzycki/data/%s/train/' % (dir)
validation_data_dir = '/datax/scratch/bbrzycki/data/%s/validation/' % (dir)
nb_train_samples = 1000*2
nb_validation_samples = 200*2
epochs = 50
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model_dir = '/datax/scratch/bbrzycki/models/scintillated/'
try:
    os.makedirs(model_dir)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
model.save_weights(model_dir + '%s_%s.h5' % ('scintillated', VERSION))

end = time.time()
print('Saved model! Took %s' % format_time(end - start))

model.summary()

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/50
31/31 [==============================] - 20s 634ms/step - loss: 0.7037 - acc: 0.5261 - val_loss: 0.6637 - val_acc: 0.6354
Epoch 2/50
31/31 [==============================] - 19s 604ms/step - loss: 0.6828 - acc: 0.6008 - val_loss: 0.6945 - val_acc: 0.5391
Epoch 3/50
31/31 [==============================] - 19s 599ms/step - loss: 0.6334 - acc: 0.6527 - val_loss: 0.5735 - val_acc: 0.6875
Epoch 4/50
31/31 [==============================] - 19s 598ms/step - loss: 0.5963 - acc: 0.7047 - val_loss: 0.5345 - val_acc: 0.6979
Epoch 5/50
31/31 [==============================] - 20s 630ms/step - loss: 0.5998 - acc: 0.7131 - val_loss: 0.5198 - val_acc: 0.7552
Epoch 6/50
31/31 [==============================] - 17s 554ms/step - loss: 0.5258 - acc: 0.7434 - val_loss: 0.5153 - val_acc: 0.7526
Epoch 7/50
31/31 [==============================] - 19s 615ms/step - loss: 0.5481 - acc: 0.7340 - val_loss: 0.5007 - va

In [17]:
# basic cnn training: 4
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

import numpy as np
import os, sys, errno
import time
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.2
# session = tf.Session(config=config)

def format_time(total):
    h = int(np.floor(total / 3600.))
    m = int(np.floor((total - 3600. * h) / 60.))
    s = (total - 3600. * h - 60. * m)
    return '%02dh %02dm %02.2fs' % (h, m, s)

start = time.time()
# dimensions of our images.
img_width, img_height = 32, 1024

VERSION = 'v4'

dir = 'scintillated_small'

train_data_dir = '/datax/scratch/bbrzycki/data/%s/train/' % (dir)
validation_data_dir = '/datax/scratch/bbrzycki/data/%s/validation/' % (dir)
nb_train_samples = 1000*2
nb_validation_samples = 200*2
epochs = 50
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model_dir = '/datax/scratch/bbrzycki/models/scintillated/'
try:
    os.makedirs(model_dir)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
model.save_weights(model_dir + '%s_%s.h5' % ('scintillated', VERSION))

end = time.time()
print('Saved model! Took %s' % format_time(end - start))

model.summary()

Found 2000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/50
62/62 [==============================] - 18s 284ms/step - loss: 0.9901 - acc: 0.5131 - val_loss: 0.6923 - val_acc: 0.4974
Epoch 2/50
62/62 [==============================] - 17s 270ms/step - loss: 0.6966 - acc: 0.5045 - val_loss: 0.7293 - val_acc: 0.4974
Epoch 3/50
62/62 [==============================] - 17s 267ms/step - loss: 0.6889 - acc: 0.5837 - val_loss: 0.6361 - val_acc: 0.5911
Epoch 4/50
62/62 [==============================] - 17s 281ms/step - loss: 0.6778 - acc: 0.6225 - val_loss: 0.5879 - val_acc: 0.6615
Epoch 5/50
62/62 [==============================] - 17s 271ms/step - loss: 0.6178 - acc: 0.6552 - val_loss: 0.5360 - val_acc: 0.7240
Epoch 6/50
62/62 [==============================] - 19s 310ms/step - loss: 0.6380 - acc: 0.6583 - val_loss: 0.5152 - val_acc: 0.7630
Epoch 7/50
62/62 [==============================] - 16s 261ms/step - loss: 0.6385 - acc: 0.7263 - val_loss: 0.5062 - va